# NYPD-Complaint-Data to DB

<span style="color: red;">**WARNING:** Only execute this notebook if your machine has sufficient memory (RAM >= 24 GB preferred)</span>

### Load `ai4sg` DB secrets

In [1]:
from dotenv import load_dotenv

import sys
import os

sys.path.append(f"..{os.path.sep}")

load_dotenv()
env_vars = {key: value for key, value in os.environ.items()}
env_vars.setdefault('MYSQL_USER', 'admin')
env_vars.setdefault('MYSQL_PWD', 'password')
env_vars.setdefault('MYSQL_HOST', 'localhost')
env_vars.setdefault('MYSQL_PORT', '3306')
env_vars.setdefault('MYSQL_DB_NAME', 'vinay')
env_vars.setdefault('MYSQL_TABLE_NAME', 'nyc_crime')
print("Loaded database secrets from '.env' file if exists!")

Loaded database secrets from '.env' file if exists!


### Loading raw data

* [NYPD Complaint Data Historic](https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i/about_data) is downloaded as a `.csv` locally and stored in `data` directory.

In [2]:
data_dir = 'data'
fname = 'NYPD_Complaint_Data_Historic_20241027.csv'
fpath = os.path.join(sys.path[-1], data_dir, fname)

assert os.path.exists(fpath), f'{os.path.abspath(fpath)} does not exists!'

In [3]:
import polars as pl

complaint_lf = pl.scan_csv(fpath, try_parse_dates=True)
complaint_lf.collect_schema()

Schema([('CMPLNT_NUM', String),
        ('CMPLNT_FR_DT', String),
        ('CMPLNT_FR_TM', String),
        ('CMPLNT_TO_DT', String),
        ('CMPLNT_TO_TM', String),
        ('ADDR_PCT_CD', Int64),
        ('RPT_DT', String),
        ('KY_CD', Int64),
        ('OFNS_DESC', String),
        ('PD_CD', Int64),
        ('PD_DESC', String),
        ('CRM_ATPT_CPTD_CD', String),
        ('LAW_CAT_CD', String),
        ('BORO_NM', String),
        ('LOC_OF_OCCUR_DESC', String),
        ('PREM_TYP_DESC', String),
        ('JURIS_DESC', String),
        ('JURISDICTION_CODE', Int64),
        ('PARKS_NM', String),
        ('HADEVELOPT', String),
        ('HOUSING_PSA', String),
        ('X_COORD_CD', Int64),
        ('Y_COORD_CD', Int64),
        ('SUSP_AGE_GROUP', String),
        ('SUSP_RACE', String),
        ('SUSP_SEX', String),
        ('TRANSIT_DISTRICT', String),
        ('Latitude', Float64),
        ('Longitude', Float64),
        ('Lat_Lon', String),
        ('PATROL_BORO', String),


### Handling datetime variables

In [4]:
complaint_lf = complaint_lf.with_columns(pl.col('CMPLNT_FR_DT').str.to_date("%m/%d/%Y"),
                                         pl.col('CMPLNT_TO_DT').str.to_date("%m/%d/%Y"),
                                         pl.col('RPT_DT').str.to_date("%m/%d/%Y").alias('report_date'),
                                         pl.when(pl.col("CMPLNT_TO_TM").str.contains('(null)'))
                                         .then(None).otherwise(pl.col('CMPLNT_TO_TM')).str.to_time("%H:%M:%S").name.keep()).drop('RPT_DT')

complaint_lf = complaint_lf.with_columns(pl.col('CMPLNT_FR_DT').dt.combine(pl.col('CMPLNT_FR_TM')).alias('cmplnt_from_date'),
                                         pl.col('CMPLNT_TO_DT').dt.combine(pl.col('CMPLNT_TO_TM')).alias('cmplnt_to_date')
                                         ).drop('CMPLNT_FR_DT','CMPLNT_FR_TM','CMPLNT_TO_DT','CMPLNT_TO_TM')

#### Checking `cmplnt_from_date` variable for dates that are not parsed correctly and bad data

In [5]:
cmplnt_from_datetime = complaint_lf.select('cmplnt_from_date').group_by('cmplnt_from_date').len('count').sort('cmplnt_from_date').collect()
cmplnt_from_datetime

cmplnt_from_date,count
datetime[μs],u32
null,702
1010-05-14 20:00:00,1
1010-08-05 13:00:00,1
1010-08-22 13:35:00,1
1010-08-29 16:20:00,1
…,…
2023-12-31 23:34:00,1
2023-12-31 23:35:00,1
2023-12-31 23:37:00,1


- In Renda's reference notebook [PrepCrimeDataForDB.ipynb](../reference/PrepCrimeDataForDB.ipynb), years that are $< 25$ are converted into years in 21st century years, whereas the data has years which are from 20th century.

In [6]:
cmplnt_from_datetime.filter(pl.col('cmplnt_from_date').dt.year().is_between(1800, 1925))

cmplnt_from_date,count
datetime[μs],u32
1900-03-10 19:00:00,1
1900-05-08 21:00:00,1
1900-06-02 19:00:00,1
1900-08-06 09:00:00,1
1900-08-07 08:30:00,1
…,…
1923-01-25 10:00:00,1
1923-04-05 17:00:00,1
1923-07-19 18:30:00,1


- The dataset does contain some bad data, in this case there are some dates from 11th century. But considering the overall size of the data these are negligible and thus can be dropped or set to `null`.

In [7]:
cmplnt_from_datetime.filter(pl.col('cmplnt_from_date').dt.year() < 1900)

cmplnt_from_date,count
datetime[μs],u32
1010-05-14 20:00:00,1
1010-08-05 13:00:00,1
1010-08-22 13:35:00,1
1010-08-29 16:20:00,1
1010-10-28 20:09:00,1
…,…
1028-12-07 09:00:00,1
1029-02-07 23:00:00,1
1029-07-04 08:30:00,1


#### Checking `cmplnt_to_date` variable for dates that are not parsed correctly and bad data

In [8]:
cmplnt_to_datetime = complaint_lf.select('cmplnt_to_date').group_by('cmplnt_to_date').len('count').sort('cmplnt_to_date').collect()
cmplnt_to_datetime

cmplnt_to_date,count
datetime[μs],u32
null,1820391
1017-12-08 12:30:00,1
1018-09-28 12:56:00,1
1023-06-28 19:15:00,1
1023-08-29 12:19:00,1
…,…
2023-12-31 23:50:00,1
2023-12-31 23:53:00,1
2023-12-31 23:56:00,2


- In Renda's reference notebook [PrepCrimeDataForDB.ipynb](../reference/PrepCrimeDataForDB.ipynb), years that are $< 25$ are converted into years in 21st century years, whereas the data has years which are from 20th century.

In [9]:
cmplnt_to_datetime.filter(pl.col('cmplnt_to_date').dt.year().is_between(1800, 1925))

cmplnt_to_date,count
datetime[μs],u32
1912-04-12 11:30:00,1
1912-05-10 18:29:00,1
1920-04-21 14:15:00,1
1920-12-30 18:07:00,1


- The dataset does contain some bad data, in this case there are some dates from 11th century. But considering the overall size of the data these are negligible and thus can be dropped or set to `null`.

In [10]:
cmplnt_to_datetime.filter(pl.col('cmplnt_to_date').dt.year() < 1900)

cmplnt_to_date,count
datetime[μs],u32
1017-12-08 12:30:00,1
1018-09-28 12:56:00,1
1023-06-28 19:15:00,1
1023-08-29 12:19:00,1


- Setting bad dates to `null`

In [11]:
complaint_lf = complaint_lf.with_columns(pl.when(pl.col('cmplnt_from_date').dt.year() < 1900).then(None).otherwise(pl.col('cmplnt_from_date')).name.keep(),
                                         pl.when(pl.col('cmplnt_to_date').dt.year() < 1900).then(None).otherwise(pl.col('cmplnt_to_date')).name.keep())

### Handling duplicate values in `CMPLNT_NUM`

In [12]:
complaint_lf.select(pl.len()).collect().item()

8914838

In [13]:
complaint_lf = complaint_lf.unique('CMPLNT_NUM')
complaint_lf.select(pl.len()).collect().item()

8913734

### Handling Latitude and Longitude variables

In [14]:
unique_lat_lon = complaint_lf.select(pl.col('Lat_Lon').unique(),
                                     pl.col('Lat_Lon').unique()
                                     .str.strip_chars('()')
                                     .str.split_exact(",",1)
                                     .struct.rename_fields(['lat','lon'])
                                     .alias("fields")).unnest("fields").with_columns(
                                         pl.col('lat').str.strip_chars(' ').cast(pl.Decimal),
                                         pl.col('lon').str.strip_chars(' ').cast(pl.Decimal)
                                         )

In [15]:
unique_lat_lon_df = unique_lat_lon.collect().to_pandas()
unique_lat_lon_df.head()

,Lat_Lon,lat,lon
0,"(40.652379, -73.889277)",40.652379000000000,-73.88927700000000
1,"(40.813953, -73.859332)",40.813953000000000,-73.85933200000000
2,"(40.732139, -73.867211)",40.732139000000000,-73.86721100000000
3,"(40.739267, -73.716938)",40.739267000000000,-73.71693800000000
4,"(40.86536643820413, -73.92438348096172)",40.865366438204130,-73.92438348096172


### Loading NYC Zip Code Data

* [Modified Zip Code Tabulation Areas (MODZCTA)](https://data.cityofnewyork.us/Health/Modified-Zip-Code-Tabulation-Areas-MODZCTA-/pri4-ifjk/about_data) is downloaded as `.geojson` locally and stored in `data` directory.

In [16]:
import geopandas

unique_lat_lon_gdf = geopandas.GeoDataFrame(unique_lat_lon_df,
                             geometry=geopandas.points_from_xy(unique_lat_lon_df.lon, unique_lat_lon_df.lat),
                             crs="EPSG:4326")

fname = 'MODZCTA.geojson'
fpath = os.path.join(sys.path[-1], data_dir, fname)

assert os.path.exists(fpath), f'{os.path.abspath(fpath)} does not exists!'
geo_df = geopandas.read_file(fpath)

Extracting zip code

In [17]:
import numpy as np

zips = np.empty(unique_lat_lon_gdf.shape[0], dtype=object)
for i, geom in enumerate(geo_df.geometry):
    zips[unique_lat_lon_gdf.within(geom)] = geo_df.modzcta[i]
zips[zips==None] = ''
unique_lat_lon_gdf['zipcode'] = zips

In [18]:
lat_lon_zip_lf = pl.from_pandas(unique_lat_lon_gdf[['Lat_Lon','zipcode']]).lazy()
lat_lon_zip_lf.collect_schema()

Schema([('Lat_Lon', String), ('zipcode', String)])

Joining the Complait data with extracted zip code data

In [19]:
complaint_lf = complaint_lf.join(lat_lon_zip_lf, on='Lat_Lon')
complaint_lf.collect_schema()

Schema([('CMPLNT_NUM', String),
        ('ADDR_PCT_CD', Int64),
        ('KY_CD', Int64),
        ('OFNS_DESC', String),
        ('PD_CD', Int64),
        ('PD_DESC', String),
        ('CRM_ATPT_CPTD_CD', String),
        ('LAW_CAT_CD', String),
        ('BORO_NM', String),
        ('LOC_OF_OCCUR_DESC', String),
        ('PREM_TYP_DESC', String),
        ('JURIS_DESC', String),
        ('JURISDICTION_CODE', Int64),
        ('PARKS_NM', String),
        ('HADEVELOPT', String),
        ('HOUSING_PSA', String),
        ('X_COORD_CD', Int64),
        ('Y_COORD_CD', Int64),
        ('SUSP_AGE_GROUP', String),
        ('SUSP_RACE', String),
        ('SUSP_SEX', String),
        ('TRANSIT_DISTRICT', String),
        ('Latitude', Float64),
        ('Longitude', Float64),
        ('Lat_Lon', String),
        ('PATROL_BORO', String),
        ('STATION_NAME', String),
        ('VIC_AGE_GROUP', String),
        ('VIC_RACE', String),
        ('VIC_SEX', String),
        ('report_date', Date),
     

- Cleaning variables with null value as '(null)' to actual `null` values.

In [20]:
import polars.selectors as cs

complaint_lf = complaint_lf.with_columns(pl.when(cs.by_dtype(pl.String).str.contains('(null)')).then(None).otherwise(cs.by_dtype(pl.String)).name.keep())

- Cast `Int64` dtypes to `Int32` in polars

In [21]:
complaint_lf = complaint_lf.with_columns(cs.by_dtype(pl.Int64).cast(pl.Int32).name.keep())

### Schema adjustments

* Existing schema for the dataset in `ai4sg` DB is given below:
```sql
CREATE TABLE `nyc_crime` (
  `id` int NOT NULL,
  `CMPLNT_NUM` text,
  `ADDR_PCT_CD` double DEFAULT NULL,
  `BORO_NM` text,
  `CRM_ATPT_CPTD_CD` text,
  `HADEVELOPT` text,
  `HOUSING_PSA` text,
  `JURISDICTION_CODE` int DEFAULT NULL,
  `JURIS_DESC` text,
  `KY_CD` int DEFAULT NULL,
  `LAW_CAT_CD` text,
  `LOC_OF_OCCUR_DESC` text,
  `OFNS_DESC` text,
  `PARKS_NM` text,
  `PATROL_BORO` text,
  `PD_CD` double DEFAULT NULL,
  `PD_DESC` text,
  `PREM_TYP_DESC` text,
  `report_date` datetime DEFAULT NULL,
  `STATION_NAME` text,
  `SUSP_AGE_GROUP` text,
  `SUSP_RACE` text,
  `SUSP_SEX` text,
  `TRANSIT_DISTRICT` text,
  `VIC_AGE_GROUP` text,
  `VIC_RACE` text,
  `VIC_SEX` text,
  `X_COORD_CD` int DEFAULT NULL,
  `Y_COORD_CD` int DEFAULT NULL,
  `Latitude` double DEFAULT NULL,
  `Longitude` double DEFAULT NULL,
  `Lat_Lon` text,
  `New Georeferenced Column` text,
  `zipcode` varchar(10) DEFAULT NULL,
  `cmplnt_from_date` datetime DEFAULT NULL,
  `cmplnt_to_date` datetime DEFAULT NULL,
  PRIMARY KEY (`id`)
);
```

In [22]:
complaint_lf.collect_schema()

Schema([('CMPLNT_NUM', String),
        ('ADDR_PCT_CD', Int32),
        ('KY_CD', Int32),
        ('OFNS_DESC', String),
        ('PD_CD', Int32),
        ('PD_DESC', String),
        ('CRM_ATPT_CPTD_CD', String),
        ('LAW_CAT_CD', String),
        ('BORO_NM', String),
        ('LOC_OF_OCCUR_DESC', String),
        ('PREM_TYP_DESC', String),
        ('JURIS_DESC', String),
        ('JURISDICTION_CODE', Int32),
        ('PARKS_NM', String),
        ('HADEVELOPT', String),
        ('HOUSING_PSA', String),
        ('X_COORD_CD', Int32),
        ('Y_COORD_CD', Int32),
        ('SUSP_AGE_GROUP', String),
        ('SUSP_RACE', String),
        ('SUSP_SEX', String),
        ('TRANSIT_DISTRICT', String),
        ('Latitude', Float64),
        ('Longitude', Float64),
        ('Lat_Lon', String),
        ('PATROL_BORO', String),
        ('STATION_NAME', String),
        ('VIC_AGE_GROUP', String),
        ('VIC_RACE', String),
        ('VIC_SEX', String),
        ('report_date', Date),
     

### Cleaned data to MySQL

Implementing `.collect()` method on the entire lazyFrame to execute all calculations and joins in sequence.

In [23]:
print(complaint_lf.explain(format='plain'))

 WITH_COLUMNS:
 [when(col("CMPLNT_NUM").str.contains([String((null))])).then(null.strict_cast(String)).otherwise(col("CMPLNT_NUM")).alias("CMPLNT_NUM"), when(col("OFNS_DESC").str.contains([String((null))])).then(null.strict_cast(String)).otherwise(col("OFNS_DESC")).alias("OFNS_DESC"), when(col("PD_DESC").str.contains([String((null))])).then(null.strict_cast(String)).otherwise(col("PD_DESC")).alias("PD_DESC"), when(col("CRM_ATPT_CPTD_CD").str.contains([String((null))])).then(null.strict_cast(String)).otherwise(col("CRM_ATPT_CPTD_CD")).alias("CRM_ATPT_CPTD_CD"), when(col("LAW_CAT_CD").str.contains([String((null))])).then(null.strict_cast(String)).otherwise(col("LAW_CAT_CD")).alias("LAW_CAT_CD"), when(col("BORO_NM").str.contains([String((null))])).then(null.strict_cast(String)).otherwise(col("BORO_NM")).alias("BORO_NM"), when(col("LOC_OF_OCCUR_DESC").str.contains([String((null))])).then(null.strict_cast(String)).otherwise(col("LOC_OF_OCCUR_DESC")).alias("LOC_OF_OCCUR_DESC"), when(col("PRE

* Sorting the data using `cmplnt_from_date` and then creating `id` column

In [24]:
complaint_lf = complaint_lf.sort('cmplnt_from_date').with_row_index('id', offset=1)
complaint_lf.head().collect()

id,CMPLNT_NUM,ADDR_PCT_CD,KY_CD,OFNS_DESC,PD_CD,PD_DESC,CRM_ATPT_CPTD_CD,LAW_CAT_CD,BORO_NM,LOC_OF_OCCUR_DESC,PREM_TYP_DESC,JURIS_DESC,JURISDICTION_CODE,PARKS_NM,HADEVELOPT,HOUSING_PSA,X_COORD_CD,Y_COORD_CD,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,Latitude,Longitude,Lat_Lon,PATROL_BORO,STATION_NAME,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,report_date,cmplnt_from_date,cmplnt_to_date,zipcode
u32,str,i32,i32,str,i32,str,str,str,str,str,str,str,i32,str,str,str,i32,i32,str,str,str,str,f64,f64,str,str,str,str,str,str,date,datetime[μs],datetime[μs],str
1,"""26098448""",110,341,"""PETIT LARCENY""",343,"""LARCENY,PETIT OF BICYCLE""","""COMPLETED""","""MISDEMEANOR""","""QUEENS""","""REAR OF""","""RESIDENCE-HOUSE""","""N.Y. POLICE DEPT""",0,null,null,null,1024195,211939,null,null,null,null,40.74831,-73.855836,"""(40.74831, -73.855836)""","""PATROL BORO QUEENS NORTH""",null,"""25-44""","""WHITE HISPANIC""","""F""",2006-12-30,null,2006-12-30 18:00:00,"""11368"""
2,"""72334535""",32,351,"""CRIMINAL MISCHIEF & RELATED OF""",259,"""CRIMINAL MISCHIEF,UNCLASSIFIED…","""COMPLETED""","""MISDEMEANOR""","""MANHATTAN""","""INSIDE""","""RESIDENCE - PUBLIC HOUSING""","""N.Y. HOUSING POLICE""",2,null,null,"""4552""",1001767,241659,null,null,null,null,40.829956,-73.936699,"""(40.829956, -73.936699)""","""PATROL BORO MAN NORTH""",null,"""25-44""","""WHITE HISPANIC""","""F""",2010-04-19,null,null,"""10039"""
3,"""81415903""",18,578,"""HARRASSMENT 2""",637,"""HARASSMENT,SUBD 1,CIVILIAN""","""COMPLETED""","""VIOLATION""","""MANHATTAN""",null,"""STREET""","""N.Y. POLICE DEPT""",0,null,null,null,987945,218576,null,"""WHITE""","""M""",null,40.766616,-73.986661,"""(40.76661622, -73.98666083)""","""PATROL BORO MAN SOUTH""",null,"""25-44""","""WHITE HISPANIC""","""M""",2011-10-25,null,2011-10-20 02:35:00,"""10019"""
4,"""59522748""",6,109,"""GRAND LARCENY""",438,"""LARCENY,GRAND FROM BUILDING (N…","""COMPLETED""","""FELONY""","""MANHATTAN""","""INSIDE""","""RESTAURANT/DINER""","""N.Y. POLICE DEPT""",0,null,null,null,984172,205736,null,null,null,null,40.731375,-74.000279,"""(40.731375, -74.000279)""","""PATROL BORO MAN SOUTH""",null,"""25-44""","""UNKNOWN""","""F""",2009-03-05,null,2009-02-18 20:00:00,"""10011"""
5,"""94379546""",73,106,"""FELONY ASSAULT""",109,"""ASSAULT 2,1,UNCLASSIFIED""","""COMPLETED""","""FELONY""","""BROOKLYN""",null,"""STREET""","""N.Y. POLICE DEPT""",0,null,null,null,1007809,179759,null,"""BLACK""","""M""",null,40.660042,-73.915086,"""(40.66004225, -73.91508647)""","""PATROL BORO BKLYN NORTH""",null,"""25-44""","""BLACK""","""M""",2014-01-09,null,null,"""11212"""


In [25]:
complaint_lf_len = complaint_lf.select(pl.len()).collect().item()
complaint_lf_len

8913268

- Breaking down the entire LazyFrame to chunks and loading the contents directly into MySQL DB. The chunk size can be increased for faster transfer speed depending on the memory available.

**Note:** The table in DB should not contain any rows prior to this operation.

In [26]:
from sqlalchemy import create_engine

# Create a SQLAlchemy engine
mysql_uri = f"mysql+pymysql://{env_vars['MYSQL_USER']}:{env_vars['MYSQL_PWD']}@{env_vars['MYSQL_HOST']}:{env_vars['MYSQL_PORT']}/{env_vars['MYSQL_DB_NAME']}"
engine = create_engine(mysql_uri)

offset = 0
chunk_size = 1000000

while offset < complaint_lf_len:
    complaint_df = complaint_lf.slice(offset, chunk_size).collect()
    complaint_df.write_database(env_vars['MYSQL_TABLE_NAME'], connection=engine, if_table_exists='append')
    offset += chunk_size